In [ ]:
import os
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt

from nibabel.orientations import axcodes2ornt, ornt_transform, inv_ornt_aff
from nibabel.affines import from_matvec, to_matvec, apply_affine
from nibabel.processing import resample_to_output, resample_from_to

## Variables

In [ ]:
dataset = 'KORA' # 'NAKO', 'UKB', 'KORANAKOUKB'
file_type = 'nifti'  # 'nerd'
target_file_type = 'nifti'
default_ornt = 'RAS'
target_res = [2,2,3]
default_view = ['Saggital', 'Coronal', 'Axial']
default_ref_view = 'Sagittal'
optimization = 'N4'  # Intensity, Min-Max, Fat-Water-Swap
is_cropping = True
default_world_coords = [500, 500, 1000]

default_output_path = './temp'


## Utilities

In [ ]:
def create_if_not(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Creating the default paths needed for smooth execution.
create_if_not(default_output_path)

def volume_viewer(vol, axis_idx=0):
    if axis_idx > 2:
        raise Exception('Axis Index cannot be more than 2! Ideally 0: Sagittal, 1: Coronal, 2: Axial.')
    axis = vol.shape
    plt.imshow(vol[axis[axis_idx]//2])
    plt.show()
    
def volume_3_view_viewer(vol):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, tight_layout=True)
    axis = vol.shape
    
    ax1.imshow(vol[axis[0]//2])
    ax2.imshow(vol[:, axis[1]//2, :])
    ax3.imshow(vol[:, :, axis[2]//2])
    
    plt.show()
    
def get_volume_data(img):
    print(f"Affine:{img.affine}, Image Shape: {img.shape}")
    return img.get_fdata()

def save_volume(img, file_name):
    nb.save(img, f'{default_output_path}/{file_name}.nii.gz')
    
def sigmoid(x):
    y = np.zeros(len(x))
    for i in range(len(x)):
        y[i] = 1 / (1 + np.exp(-x[i]))
    return y

## Volume Loader / Reader

In [ ]:
def nrrd_reader(file_path):
    _nrrd = nrrd.read(file_path)
    data = _nrrd[0]
    header = _nrrd[1]
    return data, header, None

def nibabel_reader(file_path):
    volume_nifty = nb.load(file_path)
    volume = get_volume_data(volume_nifty)
    return volume, volume_nifty.header, volume_nifty

def file_reader(file_path, file_type=None):
    header_mat = np.empty_like((4,4))
    if file_type == None:
        file_type = file_path.split('.')[-1]
    if file_type == 'nrrd':
        data, header, img = nrrd_reader(file_path)
        affine = header['space directions']
        affine = affine[:3, :3]
        origins = header['space origin']
        origins = origins[:3]
        t_mat = from_matvec(affine, origins)
        img = nb.Nifti1Image(data, t_mat) if img is None else img
        header_mat = t_mat
        
    else:
        data, header, img = nibabel_reader(file_path)
        header_mat = header
    
    return data, header_mat, img

## Transformation :- Reading Direction Optimization

In [ ]:
def do_nibabel_transform_to_ras(img):
    affine = img.affine
    orig_ornt = nb.io_orientation(affine)
    targ_ornt = axcodes2ornt('RAS')
    transform = ornt_transform(orig_ornt, targ_ornt)
    img = img.as_reoriented(transform)
    return img

def vol_stitching(images):
    # sort according to qoffset_z
    images_sorted = sorted(images, key=lambda im: im.header['qoffset_z'], reverse=True)

    # take the upper 2 scans
    im_0 = images_sorted[0]
    im_1 = images_sorted[1]

    im_0_dim_v = im_0.shape[2]
    im_1_dim_v = im_1.shape[2]

    # calculate overlap region:
    im_0_end = im_0.header['qoffset_z']
    im_1_end = im_1.header['qoffset_z']

    spacing = im_0.header['pixdim'][3]

    im_0_dim_w = im_0_dim_v * spacing
    im_1_dim_w = im_1_dim_v * spacing

    im_1_start = im_1_end + im_1_dim_w
    im_0_start = im_0_end + im_0_dim_w

    overlap = abs(im_0_end - im_1_start)

    overlap_v = int(round(overlap / spacing))

    new_im_dim = abs(round((abs(im_1_end - im_0_start)) / spacing))

    reduce_overlap_by = 0

    new_img = np.empty([im_0.shape[0], im_0.shape[1], int(new_im_dim) + reduce_overlap_by])

    im_0_data = im_0.get_fdata()
    im_1_data = im_1.get_fdata()

    overlap_v -= reduce_overlap_by

    new_img[:, :, 0:(im_1_dim_v - overlap_v)] = im_1_data[:, :, 0:(im_1_dim_v - overlap_v)]
    new_img[:, :, im_0_dim_v:] = im_0_data[:, :, overlap_v:]
    
    # overlap region:
    sigmoid_c = sigmoid(np.linspace(-30, 30, overlap_v))
    
    for l in range(0, overlap_v):
        new_img[:, :, (im_0_dim_v - overlap_v + l)] = \
        (1 - sigmoid_c[l]) * im_1_data[:,:, (im_0_dim_v - overlap_v) + l] + (sigmoid_c[l]) * im_0_data[:, :, l]
        
    empty_header = nb.Nifti1Header()
    stitched_img = nb.Nifti1Image(new_img, im_1.affine, empty_header)
    stitched_img.header['pixdim'] = im_1.header['pixdim']

    return stitched_img

## Actual Pre-Processing

In [ ]:
print("STARTING.")
print('Reading Files.....')
file_to_read1 = './datasets/kora/KORA2460249/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8.nii.gz'
file_to_read2 = './datasets/kora/KORA2460249/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz'

data1, header1, img1 = file_reader(file_to_read1, 'nifti')
data2, header2, img2 = file_reader(file_to_read2, 'nifti')

print('Viewing Raw Images.....')
volume_3_view_viewer(get_volume_data(img1))
volume_3_view_viewer(get_volume_data(img2))

print('Transforming Raw Images to RAS.....')
img1_ras = do_nibabel_transform_to_ras(img1)
img2_ras = do_nibabel_transform_to_ras(img2)

print('Viewing Transformed to RAS Images.....')
volume_3_view_viewer(get_volume_data(img1_ras))
volume_3_view_viewer(get_volume_data(img2_ras))

print('Resampling to [2,2,3] Voxel DIM and Equal Shape.....')
img1_ras_resampled = resample_to_output(img1_ras, (2,2,3), order=3, mode='constant', cval=0.0)
# img2_ras_resampled = resample_to_output(img2_ras, (2,2,3), order=3, mode='constant', cval=0.0)

target_affine = img1_ras_resampled.affine.copy()
target_affine[2,3] = img2_ras.affine[2,3].copy()
img2_ras_resampled = resample_from_to(img2_ras, [img1_ras_resampled.shape, target_affine])

print('Viewing Resampled to "Target Voxel Dim" Images.....')
volume_3_view_viewer(get_volume_data(img1_ras_resampled))
volume_3_view_viewer(get_volume_data(img2_ras_resampled))

print('Merging/Stitching Images.....')
# print(f'Upper Image Shape:{img1_ras_resampled.shape} & Affine: {img1_ras_resampled.header}')
# print(f'Lower Image Shape: {img2_ras_resampled.shape} & Affine: {img2_ras_resampled.header}')
img_ras_sampled_combined = vol_stitching([img1_ras_resampled, img2_ras_resampled])
# img_ras_sampled_combined = nb.funcs.concat_images([img1_ras_resampled, img2_ras_resampled], check_affines=True, axis=2)

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(img_ras_sampled_combined))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined, 'kora_ras_sampled_stitched')
print('FINISHED.')

In [ ]:
img = do_nibabel_transform_to_ras(img)
volume_3_view_viewer(get_volume_data(img))

out_img = resample_to_output(img, (2,2,3), order=3, mode='constant', cval=0.0)
save_volume(out_img, 'kora_processed_sampled')
volume_3_view_viewer(get_volume_data(out_img))
out_img_1 = out_img

In [ ]:
file_to_read2 = './datasets/kora/KORA2460249/t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_9.nii.gz'

data, header, img = file_reader(file_to_read2, 'nifti')

volume_3_view_viewer(get_volume_data(img))


In [ ]:
img = do_nibabel_transform_to_ras(img)
volume_3_view_viewer(get_volume_data(img))

out_img = resample_from_to(img, [out_img_1.shape, out_img_1.affine])
save_volume(out_img, 'kora_processed_sampled')
volume_3_view_viewer(get_volume_data(out_img))

out_img_2 = out_img

In [ ]:
print(f'Upper Image Shape:{out_img_1.shape}, Lower Image Shape: {out_img_2.shape}')
# out_img = nb.funcs.as_closest_canonical(out_img, enforce_diag=True)
out_img = nb.funcs.concat_images([out_img_2,out_img_1], check_affines=True, axis=2)
volume_3_view_viewer(get_volume_data(out_img))

In [ ]:
file_to_read = './datasets/nako/100017/100017_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_1.nii.gz'

data, header, img = file_reader(file_to_read)
volume_3_view_viewer(get_volume_data(img))
img = do_nibabel_transform_to_ras(img)
volume_3_view_viewer(get_volume_data(img))
save_volume(img, 'nako_processed')    

In [ ]:
file_to_read = './datasets/ukb/5331775_20201_2_0/Dixon_BH_17s_F_Dixon_BH_17s.nii.gz'

data, header, img = file_reader(file_to_read)
volume_3_view_viewer(get_volume_data(img))
img = do_nibabel_transform_to_ras(img)
volume_3_view_viewer(get_volume_data(img))
save_volume(img, 'ukb_processed')
    